In [1]:
import pandas as pd
import numpy as np
import json
import os
from tabulate import tabulate

In [2]:
import git
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

In [20]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQD\Files\RYS_CRYS\None")
cutoffs = [2,4,8,16]
potentials = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

In [21]:
converged_only = True
all_data = []

for potential in potentials:
    for cutoff in cutoffs:

        median = []
        mean = []
        min = []
        e0 = []
        e1 = []
        e2 = []
        ratios = []

        data_path = folder_path + "\\{}\\{}_{}.json".format(potential,potential, cutoff)
        
        with open(data_path, 'r') as file:
            data = json.load(file)

        num_VQD = data['num_VQD']

        converged = [all(x) for x in data['success']]
        converged_indices = [i for i, success in enumerate(converged) if success]
        converged_runs = sum(converged)

        if converged_only:
            results = [data['results'][i] for i in converged_indices]
        else:
            results = data['results']

        success_i = [i for i, success in enumerate(data['success']) if all(success)]
        successful_results = [data['results'][i] for i in success_i]
        energies =np.sort(successful_results)

        
        run_times = data['run_times']
        time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
        mean_vqe_time = np.mean(time_series)
        time_taken = np.mean(mean_vqe_time)

        for e in np.sort(energies):
            e0.append(e[0])
            e1.append(e[1])
            e2.append(e[2])

        mean_ratio = abs((np.mean(e2) - np.mean(e1)) / (np.mean(e2) - np.mean(e0)))
        median_ratio = abs((np.median(e2) - np.median(e1)) / (np.median(e2) - np.median(e0)))
        #ratios.append(r)

        e0_exact = data['exact_eigenvalues'][0]
        e1_exact = data['exact_eigenvalues'][1]
        e2_exact = data['exact_eigenvalues'][2]
        exact_ratio = abs((e2_exact - e1_exact) / (e2_exact - e0_exact))

        row = {
        "potential": potential,
        "cutoff": cutoff,
        'Converged Runs': f"{converged_runs}/100",
        'Mean VQD time (s)': f"{time_taken:.2f}",
        "e0": np.median(e0),
        "e1": np.median(e1),
        "e2": np.median(e2),
        #"mean_ratio": mean_ratio,
        #"min_ratio": np.min(ratios),
        "e0_exact": e0_exact,
        "e1_exact": e1_exact,
        "e2_exact": e2_exact,
        "median_ratio": median_ratio,
        "exact_ratio": exact_ratio
        }

        all_data.append(row)

In [22]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df

,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,e0_exact,e1_exact,e2_exact,median_ratio,exact_ratio,diff
0,AHO,2,50/100,0.30,-0.437500,-0.437500,1.015919,-0.437500,-0.437500,2.062500,1.000000,1.000000,2.321028e-08
1,AHO,4,50/100,1.25,0.254263,0.937500,2.499074,-0.164785,0.673310,1.667943,0.695637,0.542706,1.529312e-01
2,AHO,8,49/100,11.96,0.126824,2.083411,3.930937,0.032010,1.680156,1.833526,0.485665,0.085134,4.005316e-01
3,AHO,16,27/100,114.46,0.683447,2.067605,4.840409,-0.001167,1.677494,1.686381,0.667026,0.005266,6.617602e-01


In [19]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df

,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,e0_exact,e1_exact,e2_exact,median_ratio,exact_ratio,diff
0,AHO,2,50/100,0.17,-4.375000e-01,-4.374999e-01,2.062500,-0.437500,-0.437500,2.062500,1.000000e+00,1.000000e+00,2.238693e-08
1,AHO,4,50/100,0.56,2.542633e-01,6.733108e-01,1.667941,-0.164785,0.673310,1.667943,7.035763e-01,5.427061e-01,1.608702e-01
2,AHO,8,50/100,2.89,1.268245e-01,2.083413e+00,3.323420,0.032010,1.680156,1.833526,3.879151e-01,8.513391e-02,3.027812e-01
3,AHO,16,50/100,25.47,1.237370e-01,2.079098e+00,3.323416,-0.001167,1.677494,1.686381,3.888886e-01,5.266251e-03,3.836223e-01
4,QHO,2,50/100,0.18,1.926792e-12,1.962897e-07,1.000000,0.000000,0.000000,1.000000,9.999998e-01,1.000000e+00,1.962878e-07
5,QHO,4,50/100,0.31,1.968179e-08,1.000000e+00,1.000000,0.000000,1.000000,1.000000,1.440685e-07,0.000000e+00,1.440685e-07
6,QHO,8,50/100,0.54,6.176759e-09,1.000000e+00,2.000000,0.000000,1.000000,1.000000,5.000000e-01,2.220446e-16,5.000000e-01
7,QHO,16,50/100,1.31,9.164444e-09,1.000000e+00,1.500000,0.000000,1.000000,1.000000,3.333335e-01,2.220446e-16,3.333335e-01


In [8]:
df = pd.DataFrame(all_data)
df['diff'] = df[['median_ratio','exact_ratio']].apply(lambda x: np.abs(x['median_ratio'] - x['exact_ratio']), axis=1)
df

,potential,cutoff,Converged Runs,Mean VQD time (s),e0,e1,e2,e0_exact,e1_exact,e2_exact,median_ratio,exact_ratio,diff
0,AHO,2,100/100,0.64,-4.375000e-01,-4.375000e-01,-0.436246,-0.437500,-0.437500,2.062500,9.999633e-01,1.000000e+00,3.671680e-05
1,AHO,4,100/100,2.39,-1.647842e-01,6.733109e-01,1.656297,-0.164785,0.673310,1.667943,5.397817e-01,5.427061e-01,2.924375e-03
2,AHO,8,100/100,21.85,4.160560e-02,1.681011e+00,2.083244,0.032010,1.680156,1.833526,1.970148e-01,8.513391e-02,1.118809e-01
3,AHO,16,100/100,200.78,3.962557e-02,1.716024e+00,2.861201,-0.001167,1.677494,1.686381,4.058646e-01,5.266251e-03,4.005983e-01
4,DW,2,100/100,0.98,3.572331e-01,5.204717e-01,0.791603,0.357233,0.771447,1.478553,6.241943e-01,6.306019e-01,6.407598e-03
5,DW,4,100/100,5.97,9.080980e-01,1.027603e+00,1.696102,0.906560,0.950634,1.695666,8.483451e-01,9.441474e-01,9.580224e-02
6,DW,8,100/100,30.87,8.951463e-01,1.090723e+00,2.983337,0.884580,0.887725,2.693873,9.063416e-01,9.982617e-01,9.192015e-02
7,DW,16,100/100,188.93,8.961524e-01,1.091884e+00,3.144082,0.891599,0.891641,2.734122,9.129282e-01,9.999774e-01,8.704927e-02
8,QHO,2,100/100,0.85,5.135977e-10,5.296452e-08,0.046213,0.000000,0.000000,1.000000,9.999989e-01,1.000000e+00,1.134991e-06
9,QHO,4,100/100,1.45,1.255998e-07,1.000000e+00,1.000000,0.000000,1.000000,1.000000,2.001772e-07,0.000000e+00,2.001772e-07


In [19]:
latex_table = tabulate(df.values, headers=df.columns, tablefmt="latex", floatfmt=".3f")
print(latex_table)

\begin{tabular}{lrlrrrrrrrrrr}
\hline
 potential   &   cutoff & Converged Runs   &   Mean VQD time (s) &     e0 &     e1 &    e2 &   median\_ratio &   mean\_ratio &   e0\_exact &   e1\_exact &   e2\_exact &   exact\_ratio \\
\hline
 AHO         &        2 & 100/100          &              96.260 & -0.485 & -0.437 & 1.472 &          0.976 &        0.977 &     -0.437 &     -0.437 &      2.062 &         1.000 \\
 AHO         &        4 & 100/100          &             110.610 & -0.165 &  0.632 & 1.613 &          0.552 &        0.550 &     -0.165 &      0.673 &      1.668 &         0.543 \\
 AHO         &        8 & 100/100          &             152.230 &  0.032 &  1.647 & 1.796 &          0.086 &        0.089 &      0.032 &      1.680 &      1.834 &         0.085 \\
 AHO         &       16 & 97/100           &             296.420 & -0.001 &  1.652 & 1.670 &          0.017 &        0.059 &     -0.001 &      1.677 &      1.686 &         0.005 \\
 AHO         &       32 & 94/100           &